In [11]:
from torchvision import datasets
from torchvision import transforms
from matplotlib import pyplot as plt
from os.path import abspath, exists, join, dirname
from scipy import io as sio

import os, sys
import pandas as pd
import numpy as np

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Path Parameters for the training, test dataset

* DATA_HOME - Base directory for all data
* TRAIN_DIR - Path for training dataset
* TRAIN_LABEL_PATH - Path for training annotations/labels as mentioned in Stanford car dataset.
* TEST_DIR, TEST_LABEL_PATH - Similarly for test dataset

In [3]:
DATA_HOME=abspath("./../data/")
TRAIN_DIR=join(DATA_HOME, "cars_train/")
TEST_DIR=join(DATA_HOME, "cars_test/")
TRAIN_LABEL_PATH=join(DATA_HOME, "devkit/cars_train_annos.mat")
TEST_LABEL_PATH=join(DATA_HOME, "devkit/cars_test_annos_withlabels.mat")
CLASSES_PATH=join(DATA_HOME, "devkit/cars_meta.mat")

### Load the code repository

In [4]:
sys.path.append('./../grab-challenge/src/python')

In [5]:
from dataset import StanfordCarDataset, split, CarDataset
from preprocess import FilterNonRGBCarImages
from nnet import CarNet

### Class Names dictionary with its label ids

In [9]:
def get_car_classes(path) -> np.ndarray:
    carsMat = sio.loadmat(path)
    _, nclasses = carsMat["class_names"].shape
    cars_classes = np.array(
        [
            (i + 1, carsMat["class_names"][:, i][0][0])
            for i in range(nclasses)
        ]
    )
    return cars_classes

class_names = get_car_classes(CLASSES_PATH)
class_df = pd.DataFrame(class_names, columns=['class', 'class_name']).drop(columns=['class'])\
                                                        .reset_index().rename(columns={"index": "class"})
CLASS_NAMES_DICT = class_df.to_dict()['class_name']

In [10]:
NUM_CLASSES = 196
BATCH_SIZE = 64

# Code for training and evaluating the network for car's make and model

The network is build upon transfer learning technique, it uses Resnet18 which is further modified to classify NUM_CLASSES = 196 for current case. We have added the resize step so as make compatible with resnet18 input image size.

All steps are detailed below:

## Steps :

1. Data augmentation techniques - resize (224, 224), random horizontal flip and finally normalisation.

2. All labels are decreased by value 1 so as to make an adjustment with model classification output. Model will give
class ids starting with index 0 but in the dataset the labels start with 1.

3. Preprocessing step as to filter out images which does not have dimension (x, y, 3). The step is applied for both train and test dataset.

4. Load the dataset using DataLoader API of Pytorch along with custom dataset API `StanfordCarDataset`. The code for `StanfordCarDataset` API can be seen in module `dataset.py` in my repository.

5. (Imp) Initialise the `checkpoint` directory (kindly make a directory if it does not exists).
6. Initialise the `CarNet` neural network and run `fit` method using train_dataloader and val_dataloader.
7. Method `evaluate` and `predict` are used for testing the model on unseen dataset `test_loader`.

## Data Augmentation techniques using Pytorch transformations

In [17]:
train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

target_transform = transforms.Compose([
    transforms.Lambda(lambda x: x - 1)
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),\
    transforms.ToTensor(),\
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [14]:
filtered_images = FilterNonRGBCarImages(TRAIN_DIR, TRAIN_LABEL_PATH).apply()

In [18]:
train_dataset = StanfordCarDataset(TRAIN_DIR, filtered_images.rgb, transform=train_transform,\
                                   label_transform=target_transform)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

In [19]:
val_test_filtered_images = FilterNonRGBCarImages(TEST_DIR, TEST_LABEL_PATH).apply()
val_test_dataset = StanfordCarDataset(TEST_DIR, val_test_filtered_images.rgb, transform=val_test_transform,\
                                     label_transform=target_transform)

In [22]:
val_dataset, test_dataset = split(val_test_dataset, split_ratio=0.6)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [31]:
checkpoint_folder = abspath("./../checkpoint")

car_net = CarNet(NUM_CLASSES, require_chkpoint=True, chkpnt_folder=checkpoint_folder)

In [ ]:
car_net.fit(train_dataloader, val_loader, epochs=16)

Running epoch 1/16
---------------
Completed in 11m 47s training loss: 4.0904, accuracy: 0.1301
Completed in 2m 23s validation loss: 3.5577, accuracy: 0.1672
Running epoch 2/16
---------------
Completed in 11m 46s training loss: 2.1554, accuracy: 0.4331
Completed in 2m 23s validation loss: 2.5487, accuracy: 0.3342
Running epoch 3/16
---------------
Completed in 11m 47s training loss: 1.2512, accuracy: 0.6526
Completed in 2m 28s validation loss: 2.2451, accuracy: 0.4224
Running epoch 4/16
---------------
Completed in 11m 45s training loss: 0.7768, accuracy: 0.7765
Completed in 2m 24s validation loss: 1.7736, accuracy: 0.5261
Running epoch 5/16
---------------
Completed in 11m 47s training loss: 0.4822, accuracy: 0.8645
Completed in 2m 24s validation loss: 1.4367, accuracy: 0.6147
Running epoch 6/16
---------------
Completed in 11m 46s training loss: 0.3028, accuracy: 0.9159
Completed in 2m 24s validation loss: 1.4517, accuracy: 0.6118
Running epoch 7/16
---------------
Completed in 11m 